In [1]:
%%html
<style>
table {align:left;display:block}  # to align html tables to left
</style>

In [2]:
# import libraries
import pandas as pd  # Dataframe manipulation
import numpy as np  # Arrays manipulation 
import matplotlib.pyplot as plt  # Plot charts
import matplotlib.style as style  # Inherit styles

# Enables Jupyter to display graphs
%matplotlib inline

# Dataquest - Data Cleaning And Analysis <br/> <br/> Project Title: Clean And Analyse Employee Exit Surveys

## 1) Introduction

#### Key skills applied in project:
- Explore the data and figure out how to prepare it for analysis
- Correct some of the missing values
- Drop any data not needed for our analysis
- Rename our columns
- Verify the quality of data
- Create new columns (eg. feature engineering)
- Combine the data
- Handle the missing values in other column
- Aggregate the data
- Example tools for data cleaning: 
  - apply(), map(), applymap(), fillna(), dropna(), drop(), melt(), concat(), merge()

#### Background
Provided by: [Dataquest.io](https://www.dataquest.io/)

We'll work with exit surveys from employees of the [Department of Education, Training and Employment](https://en.wikipedia.org/wiki/Department_of_Education_and_Training_(Queensland)) (DETE) and the Technical and Further Education (TAFE) institute in Queensland, Australia. The original TAFE exit survey here and the survey for the DETE are located [here](https://data.gov.au/dataset/ds-qld-89970a3b-182b-41ea-aea2-6f9f17b5907e/details?q=exit%20survey). Dataquest.io had made some slight modifications to these datasets to make them easier to work with, including changing the encoding to **UTF-8** (the original ones are encoded using **cp1252**.) (Modified files are located in same GitHub folder of this jupyter notebook.)

In this project, we'll play the role of data analyst and pretend our stakeholders want to know the following:
- Are employees who only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction? What about employees who have been there longer?
- Are younger employees resigning due to some kind of dissatisfaction? What about older employees?

They want us to combine the results for both surveys to answer these questions. However, although both used the same survey template, one of them customized some of the answers.

A data dictionary wasn't provided with the dataset. In a job setting, we'd make sure to meet with a manager to confirm the definitions of the data. For this project, we'll use our general knowledge to define the columns.

Below is a preview of a couple columns we'll work with from the **dete_survey.csv**:

Attribute | Description
--- | ---
ID | An id used to identify the participant of the survey
SeparationType | The reason why the person's employment ended
Cease Date | The year or month the person's employment ended
DETE Start Date | The year the person began employment with the DETE

Below is a preview of a couple columns we'll work with from the **tafe_survey.csv**:

Attribute | Description
--- | ---
Record ID | An id used to identify the participant of the survey
Reason for ceasing employment | The reason why the person's employment ended
LengthofServiceOverall. Overall Length of Service at Institute (in years) | The length of the person's employment (in years)

Let's start by reading the datasets into pandas and exploring them.

In [3]:
# read and load files
dete_survey = pd.read_csv('dete_survey.csv')
tafe_survey = pd.read_csv('tafe_survey.csv')

# review dete files
print(dete_survey.info())  # review columns names and dtype
print(dete_survey.isnull().sum())  # review any null values
dete_survey.head(3)  # review first few rows and values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822 entries, 0 to 821
Data columns (total 56 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   ID                                   822 non-null    int64 
 1   SeparationType                       822 non-null    object
 2   Cease Date                           822 non-null    object
 3   DETE Start Date                      822 non-null    object
 4   Role Start Date                      822 non-null    object
 5   Position                             817 non-null    object
 6   Classification                       455 non-null    object
 7   Region                               822 non-null    object
 8   Business Unit                        126 non-null    object
 9   Employment Status                    817 non-null    object
 10  Career move to public sector         822 non-null    bool  
 11  Career move to private sector        822 non-

,ID,SeparationType,Cease Date,DETE Start Date,Role Start Date,Position,Classification,Region,Business Unit,Employment Status,...,Kept informed,Wellness programs,Health & Safety,Gender,Age,Aboriginal,Torres Strait,South Sea,Disability,NESB
0,1,Ill Health Retirement,08/2012,1984,2004,Public Servant,A01-A04,Central Office,Corporate Strategy and Peformance,Permanent Full-time,...,N,N,N,Male,56-60,NaN,NaN,NaN,NaN,Yes
1,2,Voluntary Early Retirement (VER),08/2012,Not Stated,Not Stated,Public Servant,AO5-AO7,Central Office,Corporate Strategy and Peformance,Permanent Full-time,...,N,N,N,Male,56-60,NaN,NaN,NaN,NaN,NaN
2,3,Voluntary Early Retirement (VER),05/2012,2011,2011,Schools Officer,NaN,Central Office,Education Queensland,Permanent Full-time,...,N,N,N,Male,61 or older,NaN,NaN,NaN,NaN,NaN


In [4]:
# review tafe files
print(tafe_survey.info())  # review columns names and dtype
print(tafe_survey.isnull().sum())  # review any null values
tafe_survey.head(3)  # review first few rows and values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 72 columns):
 #   Column                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                         --------------  -----  
 0   Record ID                                                                                                                                                      702 non-null    float64
 1   Institute                                                                                                                                                      702 non-null    object 
 2   WorkArea                                                                                                                                  

,Record ID,Institute,WorkArea,CESSATION YEAR,Reason for ceasing employment,Contributing Factors. Career Move - Public Sector,Contributing Factors. Career Move - Private Sector,Contributing Factors. Career Move - Self-employment,Contributing Factors. Ill Health,Contributing Factors. Maternity/Family,...,Workplace. Topic:Does your workplace promote a work culture free from all forms of unlawful discrimination?,Workplace. Topic:Does your workplace promote and practice the principles of employment equity?,Workplace. Topic:Does your workplace value the diversity of its employees?,Workplace. Topic:Would you recommend the Institute as an employer to others?,Gender. What is your Gender?,CurrentAge. Current Age,Employment Type. Employment Type,Classification. Classification,LengthofServiceOverall. Overall Length of Service at Institute (in years),LengthofServiceCurrent. Length of Service at current workplace (in years)
0,6.341330e+17,Southern Queensland Institute of TAFE,Non-Delivery (corporate),2010.0,Contract Expired,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,Yes,Yes,Female,26 30,Temporary Full-time,Administration (AO),1-2,1-2
1,6.341337e+17,Mount Isa Institute of TAFE,Non-Delivery (corporate),2010.0,Retirement,-,-,-,-,-,...,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
2,6.341388e+17,Mount Isa Institute of TAFE,Delivery (teaching),2010.0,Retirement,-,-,-,-,-,...,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN


## 2) Identify Missing Values And Drop Unnecessary Columns

We identified the following observations:
- The **dete_survey** dataframe contains **'Not Stated'** values that indicate values are missing, but they aren't represented as **NaN**.
- Both the **dete_survey** and **tafe_survey** dataframes contain many columns that we don't need to complete our analysis.
- Each dataframe contains many of the same columns, but the column names are different.
- There are multiple columns/answers that indicate an employee resigned because they were dissatisfied.

To start, we'll handle the first two issues. We can use the **pd.read_csv()** function to specify values that should be represented as **NaN**. We'll use this function to fix the missing values first. Then, we'll drop columns we know we don't need for our analysis.

In [5]:
# pd.read_csv - na_values: Additional strings to recognize as NA/NaN.
dete_survey = pd.read_csv('dete_survey.csv', na_values='Not Stated')

In [6]:
# date survey
# drop unneeded columns and assign to new dataframe
dete_survey_updated = dete_survey.drop(dete_survey.columns[28:49], axis=1)

# review columns dropped
dete_survey.columns[28:49]

Index(['Professional Development', 'Opportunities for promotion',
       'Staff morale', 'Workplace issue', 'Physical environment',
       'Worklife balance', 'Stress and pressure support',
       'Performance of supervisor', 'Peer support', 'Initiative', 'Skills',
       'Coach', 'Career Aspirations', 'Feedback', 'Further PD',
       'Communication', 'My say', 'Information', 'Kept informed',
       'Wellness programs', 'Health & Safety'],
      dtype='object')

In [7]:
# tafe survey
# drop unneeded columns and assign to new dataframe
tafe_survey_updated = tafe_survey.drop(tafe_survey.columns[17:66], axis=1)

# review columns dropped
tafe_survey.columns[17:66]

Index(['Main Factor. Which of these was the main factor for leaving?',
       'InstituteViews. Topic:1. I feel the senior leadership had a clear vision and direction',
       'InstituteViews. Topic:2. I was given access to skills training to help me do my job better',
       'InstituteViews. Topic:3. I was given adequate opportunities for personal development',
       'InstituteViews. Topic:4. I was given adequate opportunities for promotion within %Institute]Q25LBL%',
       'InstituteViews. Topic:5. I felt the salary for the job was right for the responsibilities I had',
       'InstituteViews. Topic:6. The organisation recognised when staff did good work',
       'InstituteViews. Topic:7. Management was generally supportive of me',
       'InstituteViews. Topic:8. Management was generally supportive of my team',
       'InstituteViews. Topic:9. I was kept informed of the changes in the organisation which would affect me',
       'InstituteViews. Topic:10. Staff morale was positive w

## 3) Clean Column Names

Each dataframe contains many of the same columns, but the column names are different.

Because we eventually want to combine them, we'll have to standardize the column names.

Below are some of the columns we'd like to use for our final analysis:


dete_survey | tafe_survey | Definition
--- | --- | ---
ID | Record ID | An id used to identify the participant of the survey
SeparationType | Reason for ceasing employment | The reason why the participant's employment ended
Cease Date | CESSATION YEAR | The year or month the participant's employment ended
DETE Start Date |  | The year the participant began employment with the DETE 
 | LengthofServiceOverall. Overall Length of Service at Institute (in years) | The length of the person's employment (in years)
Age | CurrentAge. Current Age | The age of the participant
Gender | Gender. What is your Gender? | The gender of the participant



In [8]:
# Rename the remaining columns in the dete_survey_updated dataframe.

#.str.lower() - lowercase
#.str.strip() - trim white space around element
#.str.replace() - replace string, \s is regex pattern to search for single white space, regex=True to enable the usage of regex
dete_survey_updated.columns = dete_survey_updated.columns.str.lower().str.strip().str.replace('\s', '_', regex=True)

# review transformation
dete_survey_updated.columns

Index(['id', 'separationtype', 'cease_date', 'dete_start_date',
       'role_start_date', 'position', 'classification', 'region',
       'business_unit', 'employment_status', 'career_move_to_public_sector',
       'career_move_to_private_sector', 'interpersonal_conflicts',
       'job_dissatisfaction', 'dissatisfaction_with_the_department',
       'physical_work_environment', 'lack_of_recognition',
       'lack_of_job_security', 'work_location', 'employment_conditions',
       'maternity/family', 'relocation', 'study/travel', 'ill_health',
       'traumatic_incident', 'work_life_balance', 'workload',
       'none_of_the_above', 'gender', 'age', 'aboriginal', 'torres_strait',
       'south_sea', 'disability', 'nesb'],
      dtype='object')

In [9]:
# use df.rename() to rename columns in tafe_survey_updated dataset

mapping = {'Record ID': 'id',
'CESSATION YEAR': 'cease_date',
'Reason for ceasing employment': 'separationtype',
'Gender. What is your Gender?': 'gender',
'CurrentAge. Current Age': 'age',
'Employment Type. Employment Type': 'employment_status',
'Classification. Classification': 'position',
'LengthofServiceOverall. Overall Length of Service at Institute (in years)': 'institute_service',
'LengthofServiceCurrent. Length of Service at current workplace (in years)': 'role_service'}

tafe_survey_updated.rename(mapper=mapping, axis=1, inplace=True)  # axis=1 to modify columns
tafe_survey_updated.columns

Index(['id', 'Institute', 'WorkArea', 'cease_date', 'separationtype',
       'Contributing Factors. Career Move - Public Sector ',
       'Contributing Factors. Career Move - Private Sector ',
       'Contributing Factors. Career Move - Self-employment',
       'Contributing Factors. Ill Health',
       'Contributing Factors. Maternity/Family',
       'Contributing Factors. Dissatisfaction',
       'Contributing Factors. Job Dissatisfaction',
       'Contributing Factors. Interpersonal Conflict',
       'Contributing Factors. Study', 'Contributing Factors. Travel',
       'Contributing Factors. Other', 'Contributing Factors. NONE', 'gender',
       'age', 'employment_status', 'position', 'institute_service',
       'role_service'],
      dtype='object')

## 4) Filter The Data

Let's remove more of the data we don't need.

Our end goal is to answer the following question:

- Are employees who have only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction? What about employees who have been at the job longer?

If we look at the unique values in the separationtype columns in each dataframe, we'll see that each contains a couple of different separation types. For this project, we'll only analyze survey respondents who resigned, so their separation type contains the string 'Resignation'.

**dete_survey_updated** dataframe contains multiple separation types with the string 'Resignation':

- Resignation-Other reasons
- Resignation-Other employer
- Resignation-Move overseas/interstate

We'll have to account for each of these variations so we don't unintentionally drop data.

In [10]:
# Use the Series.value_counts() method to review the unique values in the separationtype column in both dete and tafe datasets

dete_survey_updated['separationtype'].value_counts()

Age Retirement                          285
Resignation-Other reasons               150
Resignation-Other employer               91
Resignation-Move overseas/interstate     70
Voluntary Early Retirement (VER)         67
Ill Health Retirement                    61
Other                                    49
Contract Expired                         34
Termination                              15
Name: separationtype, dtype: int64

In [11]:
tafe_survey_updated['separationtype'].value_counts()

Resignation                 340
Contract Expired            127
Retrenchment/ Redundancy    104
Retirement                   82
Transfer                     25
Termination                  23
Name: separationtype, dtype: int64

In [12]:
# In each dataframe, select only the data for survey respondents who have a Resignation separation type. Use df.copy() to avoid changing the initial dataset by accident.

# regex=True to enable regex searching:
# r'Resignation': regex pattern search for string that contains 'Resignation'
dete_resignations = dete_survey_updated[dete_survey_updated['separationtype'].str.contains(r'Resignation', regex=True)].copy()

dete_resignations

,id,separationtype,cease_date,dete_start_date,role_start_date,position,classification,region,business_unit,employment_status,...,work_life_balance,workload,none_of_the_above,gender,age,aboriginal,torres_strait,south_sea,disability,nesb
3,4,Resignation-Other reasons,05/2012,2005.0,2006.0,Teacher,Primary,Central Queensland,NaN,Permanent Full-time,...,False,False,False,Female,36-40,NaN,NaN,NaN,NaN,NaN
5,6,Resignation-Other reasons,05/2012,1994.0,1997.0,Guidance Officer,NaN,Central Office,Education Queensland,Permanent Full-time,...,False,False,False,Female,41-45,NaN,NaN,NaN,NaN,NaN
8,9,Resignation-Other reasons,07/2012,2009.0,2009.0,Teacher,Secondary,North Queensland,NaN,Permanent Full-time,...,False,False,False,Female,31-35,NaN,NaN,NaN,NaN,NaN
9,10,Resignation-Other employer,2012,1997.0,2008.0,Teacher Aide,NaN,NaN,NaN,Permanent Part-time,...,False,False,False,Female,46-50,NaN,NaN,NaN,NaN,NaN
11,12,Resignation-Move overseas/interstate,2012,2009.0,2009.0,Teacher,Secondary,Far North Queensland,NaN,Permanent Full-time,...,False,False,False,Male,31-35,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,810,Resignation-Other reasons,12/2013,2010.0,2010.0,Teacher Aide,NaN,NaN,NaN,Permanent Part-time,...,False,False,False,Female,26-30,NaN,NaN,NaN,NaN,NaN
815,817,Resignation-Other employer,01/2014,2012.0,2012.0,Teacher,Primary,Far North Queensland,NaN,Permanent Full-time,...,False,False,False,Male,21-25,NaN,NaN,NaN,NaN,NaN
816,818,Resignation-Move overseas/interstate,01/2014,2012.0,2012.0,Teacher,Secondary,North Coast,NaN,Permanent Full-time,...,False,False,False,Female,21-25,NaN,NaN,NaN,NaN,NaN
819,821,Resignation-Move overseas/interstate,01/2014,2009.0,2009.0,Public Servant,A01-A04,Central Office,Education Queensland,Permanent Full-time,...,True,False,False,Female,31-35,NaN,NaN,NaN,NaN,NaN


In [13]:
# na=False turns any NaN value flagged to be False
tafe_resignations = tafe_survey_updated[tafe_survey_updated['separationtype'].str.contains('Resignation', na=False)].copy()

tafe_resignations

,id,Institute,WorkArea,cease_date,separationtype,Contributing Factors. Career Move - Public Sector,Contributing Factors. Career Move - Private Sector,Contributing Factors. Career Move - Self-employment,Contributing Factors. Ill Health,Contributing Factors. Maternity/Family,...,Contributing Factors. Study,Contributing Factors. Travel,Contributing Factors. Other,Contributing Factors. NONE,gender,age,employment_status,position,institute_service,role_service
3,6.341399e+17,Mount Isa Institute of TAFE,Non-Delivery (corporate),2010.0,Resignation,-,-,-,-,-,...,-,Travel,-,-,NaN,NaN,NaN,NaN,NaN,NaN
4,6.341466e+17,Southern Queensland Institute of TAFE,Delivery (teaching),2010.0,Resignation,-,Career Move - Private Sector,-,-,-,...,-,-,-,-,Male,41 45,Permanent Full-time,Teacher (including LVT),3-4,3-4
5,6.341475e+17,Southern Queensland Institute of TAFE,Delivery (teaching),2010.0,Resignation,-,-,-,-,-,...,-,-,Other,-,Female,56 or older,Contract/casual,Teacher (including LVT),7-10,7-10
6,6.341520e+17,Barrier Reef Institute of TAFE,Non-Delivery (corporate),2010.0,Resignation,-,Career Move - Private Sector,-,-,Maternity/Family,...,-,-,Other,-,Male,20 or younger,Temporary Full-time,Administration (AO),3-4,3-4
7,6.341537e+17,Southern Queensland Institute of TAFE,Delivery (teaching),2010.0,Resignation,-,-,-,-,-,...,-,-,Other,-,Male,46 50,Permanent Full-time,Teacher (including LVT),3-4,3-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,6.350660e+17,Southern Queensland Institute of TAFE,Non-Delivery (corporate),2013.0,Resignation,-,Career Move - Private Sector,-,-,-,...,-,-,-,-,Male,21 25,Temporary Full-time,Operational (OO),5-6,5-6
697,6.350668e+17,Barrier Reef Institute of TAFE,Delivery (teaching),2013.0,Resignation,Career Move - Public Sector,-,-,-,-,...,-,-,-,-,Male,51-55,Temporary Full-time,Teacher (including LVT),1-2,1-2
698,6.350677e+17,Southern Queensland Institute of TAFE,Non-Delivery (corporate),2013.0,Resignation,Career Move - Public Sector,-,-,-,-,...,-,-,-,-,NaN,NaN,NaN,NaN,NaN,NaN
699,6.350704e+17,Tropical North Institute of TAFE,Delivery (teaching),2013.0,Resignation,-,-,-,-,-,...,-,-,Other,-,Female,51-55,Permanent Full-time,Teacher (including LVT),5-6,1-2


## 5) Verify The Data

Let's verify that the data doesn't contain any major inconsistencies (to the best of our knowledge). It may not always be possible to catch all of these errors, but by making sure the data seems reasonable to the best of our knowledge, we can stop ourselves from completing a data analysis project that winds up being useless because of bad data.

In this step, we'll focus on verifying that the years in the **cease_date** and **dete_start_date columns** make sense.

- Since the **cease_date** is the last year of the person's employment and the **dete_start_date** is the person's first year of employment, it wouldn't make sense to have years after the current date.

- Given that most people start working in their 20s, it's also unlikely that the **dete_start_date** was before the year 1940.

If we have many years higher than the current date or lower than 1940, we wouldn't want to continue with our analysis, because it could mean there's something very wrong with the data. If there are a small amount of values that are unrealistically high or low, we can remove them.

In [14]:
# Check the years in each dataframe for logical inconsistencies.


# review dete 'cease_date' values
# ignore rows with NaN values for now
dete_resignations['cease_date'].value_counts(dropna=False)

2012       126
2013        74
01/2014     22
12/2013     17
06/2013     14
09/2013     11
NaN         11
07/2013      9
11/2013      9
10/2013      6
08/2013      4
05/2013      2
05/2012      2
2010         1
07/2006      1
07/2012      1
09/2010      1
Name: cease_date, dtype: int64

In [15]:
# data-cleaning: extract the year to create new attribute
dete_resignations['cease_date_year'] = dete_resignations['cease_date'].str.extract(r'(20[0-9][0-9])')

# convert values to float to be standardise with tafe dataset
dete_resignations['cease_date_year'] = dete_resignations['cease_date_year'].astype(dtype='float')

In [22]:
# check transformation
print(dete_resignations['cease_date_year'].dtype)
dete_resignations['cease_date_year'].value_counts(dropna=False).sort_index()

float64


2006.0      1
2010.0      2
2012.0    129
2013.0    146
2014.0     22
NaN        11
Name: cease_date_year, dtype: int64

In [21]:
# review tafe 'cease_date' values
print(tafe_resignations['cease_date'].dtype)
tafe_resignations['cease_date'].value_counts(dropna=False).sort_index()

float64


2009.0      2
2010.0     68
2011.0    116
2012.0     94
2013.0     55
NaN         5
Name: cease_date, dtype: int64

In [18]:
# convert tafe 'cease_date' to be float, to be consistent with dete's 'cease_date'

tafe_resignations['cease_date_year'] = tafe_resignations['cease_date'].astype(dtype='float')
print(tafe_resignations['cease_date'].dtype)

float64


#### Findings (Verify The Data):

1) There aren't any major issues with the years.
2) The years in each dataframe don't span quite the same number of years. We'll assess on the need to drop any years not needed for the analysis.

## 6) Create A New Column

Now that we've verified the years in the **dete_resignations** dataframe, we'll use them to create a new column. Recall that our end goal is to answer the following question:

- Are employees who have only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction? What about employees who have been at the job longer?

In the Human Resources field, the length of time an employee spent in a workplace is referred to as their years of service.

The **tafe_resignations** dataframe already contains a "service" column, which we renamed to **institute_service**. In order to analyze both surveys together, we'll have to create a corresponding **institute_service** column in **dete_resignations**.

In [28]:
# create new attribute column to calculate number of years of service; ie. feature engineering
dete_resignations['institute_service'] = dete_resignations['cease_date_year'] - dete_resignations['dete_start_date']

# review transformations
# ignore NaN values for now
dete_resignations['institute_service'].value_counts(dropna=False).sort_index()

0.0     20
1.0     22
2.0     14
3.0     20
4.0     16
5.0     23
6.0     17
7.0     13
8.0      8
9.0     14
10.0     6
11.0     4
12.0     6
13.0     8
14.0     6
15.0     7
16.0     5
17.0     6
18.0     5
19.0     3
20.0     7
21.0     3
22.0     6
23.0     4
24.0     4
25.0     2
26.0     2
27.0     1
28.0     2
29.0     1
30.0     2
31.0     1
32.0     3
33.0     1
34.0     1
35.0     1
36.0     2
38.0     1
39.0     3
41.0     1
42.0     1
49.0     1
NaN     38
Name: institute_service, dtype: int64

## 7) Identify Dissatisfied Employees

We had created a new **institute_service** column that we'll use to analyze survey respondents according to their length of employment. Next, we'll identify any employees who resigned because they were dissatisfied.

Below are the columns we'll use to categorize employees as "dissatisfied" from each dataframe, based on understanding of the qualitative factors.

**tafe_survey_updated**:
- Contributing Factors. Dissatisfaction
- Contributing Factors. Job Dissatisfaction

**dete_survey_updated**:
- job_dissatisfaction
- dissatisfaction_with_the_department
- physical_work_environment
- lack_of_recognition
- lack_of_job_security
- work_location
- employment_conditions
- work_life_balance
- workload

If the employee indicated any of the factors above caused them to resign, we'll mark them as **dissatisfied** in a new column.

After our changes, the new **dissatisfied** column would contain just the following values:

- **True**: indicates a person resigned because they were dissatisfied with the job
- **False**: indicates a person resigned because of a reason other than dissatisfaction with the job
- **NaN**: indicates the value is missing

In [33]:
# working on tafe dataset


# review unique values of tate's columns that we are focusing on
print(tafe_resignations['Contributing Factors. Dissatisfaction'].value_counts())
print('')
print(tafe_resignations['Contributing Factors. Job Dissatisfaction'].value_counts())

-                                         277
Contributing Factors. Dissatisfaction      55
Name: Contributing Factors. Dissatisfaction, dtype: int64

-                      270
Job Dissatisfaction     62
Name: Contributing Factors. Job Dissatisfaction, dtype: int64
